In [2]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

In [8]:
charEntries = " 0123456789abcdefghijklmnopqrstuvwxyzäöåABCDEFGHIJKLMNOPQRSTUVWXYZÄÖÅ" + \
                ".,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|\u2013\u2014\u201C\u201D"

casingEntries = ['PADDING', 'other', 'numeric', 'mainly_numeric', 'allLower',
                   'allUpper', 'mainly_allUpper', 'initialUpper', 'contains_upper',
                   'contains_digit']
charEmbedding = 'cnn'


def dummy_generator(n=1000, length=20):
    for _ in range(1000):
        dummy_data = []
        for i in range(length):
            token={'lemma': int(np.random.uniform(1,1000)), 'surface': 'dummy_'+str(i), 'label': int(np.random.uniform(0,10))}
            dummy_data.append(token)
        yield dummy_data

next(dummy_generator())

[{'lemma': 598, 'surface': 'dummy_0', 'label': 8},
 {'lemma': 606, 'surface': 'dummy_1', 'label': 3},
 {'lemma': 115, 'surface': 'dummy_2', 'label': 0},
 {'lemma': 495, 'surface': 'dummy_3', 'label': 1},
 {'lemma': 597, 'surface': 'dummy_4', 'label': 8},
 {'lemma': 958, 'surface': 'dummy_5', 'label': 0},
 {'lemma': 717, 'surface': 'dummy_6', 'label': 5},
 {'lemma': 579, 'surface': 'dummy_7', 'label': 8},
 {'lemma': 268, 'surface': 'dummy_8', 'label': 9},
 {'lemma': 899, 'surface': 'dummy_9', 'label': 2},
 {'lemma': 948, 'surface': 'dummy_10', 'label': 6},
 {'lemma': 675, 'surface': 'dummy_11', 'label': 3},
 {'lemma': 887, 'surface': 'dummy_12', 'label': 7},
 {'lemma': 48, 'surface': 'dummy_13', 'label': 9},
 {'lemma': 808, 'surface': 'dummy_14', 'label': 6},
 {'lemma': 840, 'surface': 'dummy_15', 'label': 1},
 {'lemma': 329, 'surface': 'dummy_16', 'label': 9},
 {'lemma': 328, 'surface': 'dummy_17', 'label': 5},
 {'lemma': 925, 'surface': 'dummy_18', 'label': 9},
 {'lemma': 238, 'surfac

In [38]:
sentence_length= tf.placeholder(tf.int32, [None], name='sentence_length')

tokens_input = tf.placeholder(tf.int32, [None, None], name='words_input')
#W = tf.Variable(tf.constant(0.0, shape=[len(self.word2Idx), len(self.embeddings)], name="W_token"), trainable=False)
W = tf.Variable(tf.random_uniform([10000, 300], -1.0, 1.0), name="W_char")
embeddings = tf.placeholder(tf.float32, [10000, 300])
W.assign(embeddings)
tokens = tf.nn.embedding_lookup(W, tokens_input, name='tokens')
print(tokens.shape)

casing_input = tf.placeholder(tf.int32, [None, None],name='casing_input')
W = tf.Variable(tf.random_uniform([len(casingEntries), 30], -1.0, 1.0), name="W_case")
casings = tf.nn.embedding_lookup(W, casing_input, name='casings')
print(casings.shape)

chars_input = tf.placeholder(tf.int32, [None, None, 50], name='char_input')
W = tf.Variable(tf.random_uniform([len(charEntries), 300], -1.0, 1.0), name="W_char")
chars = tf.nn.embedding_lookup(W, chars_input, name='char_emd')
print(chars.shape)
if charEmbedding== 'lstm':
    #chars = tf.boolean_mask(chars, np.array(chars[-1] != 1))
    chars = tf.reshape(casings, [-1, chars_shape[-1], 50])
    lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(30)
    lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(30)
    (output_fw, output_bw), _ = \
        tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, chars, dtype=tf.float32, name="char_lstm")
    chars = tf.concat([output_fw, output_bw], axis=-1)
    
else:
    chars = tf.layers.Conv1D(30, 30, padding='same', name='char_cnn')(chars)
    chars = tf.layers.MaxPooling1D(50, strides=1, name="char_pooling")(chars)
    chars = tf.reshape(casings, [-1, chars_shape[-1], 30])
    #chars = tf.boolean_mask(chars, np.array(chars[-1] != 0))

y_true = tf.placeholder(tf.int32, [None, 1])
input_nodes = [tokens, casings, chars]
merged_input = tf.concat([_[1] for _ in input_nodes], axis=1)

merged = merged_input
cnt = 1
for size in (100,100):
    lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(size)
    lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(size)
    if isinstance((0.25,0.25), (list, tuple)):
        lstm_fw_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_fw_cell,
                                                     input_keep_prob=1 - 0.25,
                                                     output_keep_prob=1 - 0.25)
        lstm_bw_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_bw_cell,
                                                     input_keep_prob=1 - 0.25,
                                                     output_keep_prob=1 - 0.25)
        (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, merged,
                                       dtype=tf.float32)
        merged = tf.concat([output_fw, output_bw], axis=-1)
    cnt += 1
merged = tf.reshape(merged_input, [-1, s[-1], merged.shape[-1]])








(?, ?, 300)
(?, ?, 30)
(?, ?, 50, 300)


ValueError: Input 0 of layer char_cnn is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: [None, None, 50, 300]

In [17]:
merged_input

<tf.Tensor 'concat_3:0' shape=(1, 360) dtype=float32>